In [111]:
import pandas as pd
import torch 
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision 
import torchvision.transforms as transforms
import numpy as np

import matplotlib.pyplot as plt
from PIL import Image

import requests
import json
import ast
import os

import difflib

### Comparing Classes from Caltech101 dataset and ImageNet

In [17]:
# --- get dictionary for image net
imagenet_dct_url = "https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a/raw"
resp = requests.get(imagenet_dct_url, verify=False)
dct_imagenet = ast.literal_eval(resp.text)

/Users/emulie/pvenv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gist.github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/emulie/pvenv/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gist.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [7]:
# --- get classes names for caltech101
with open('../caltech-dct.txt', 'r') as f:
    classes_caltech101 = f.readlines()
classes_caltech101 = [name.strip().lower() for name in classes_caltech101]

In [9]:
classes_caltech101[:5]

['faces', 'faces_easy', 'leopards', 'motorbikes', 'accordion']

In [21]:
classes_imagenet = [name.lower() for name in dct_imagenet.values()]
classes_imagenet[:5]

['tench, tinca tinca',
 'goldfish, carassius auratus',
 'great white shark, white shark, man-eater, man-eating shark, carcharodon carcharias',
 'tiger shark, galeocerdo cuvieri',
 'hammerhead, hammerhead shark']

In [22]:
# --- compute overlapping classes
classes_imagenet_split = [classe.split(', ') for classe in classes_imagenet]
classes_imagenet_split = [[word.replace(' ', '_') for word in classe] for classe in classes_imagenet_split]
classes_imagenet_split[:5]

In [62]:
classes_imagenet_flat_lst = [word for classe in classes_imagenet for word in classe]

In [80]:
classes_imagenet_flat_lst2 = [word for classe in classes_imagenet for word in classe.replace(',', '').split(' ')]
classes_imagenet_flat_lst2[:5]

['tench', 'tinca', 'tinca', 'goldfish', 'carassius']

In [75]:
missing_classes = []
matching_classes = []
for caltech101_class in classes_caltech101:
    closest_classes = difflib.get_close_matches(caltech101_class, classes_imagenet_flat_lst2, n=1, cutoff=0.8)
    
    if len(closest_classes) == 0:
        missing_classes.append(caltech101_class)
    else:
        matching_classes.append((caltech101_class, closest_classes))
        

In [77]:
matching_classes

[('faces', ['face']),
 ('leopards', ['leopard']),
 ('accordion', ['accordion']),
 ('airplanes', ['warplane']),
 ('ant', ['ant']),
 ('barrel', ['barrel']),
 ('bass', ['brass']),
 ('beaver', ['beaver']),
 ('binocular', ['binoculars']),
 ('bonsai', ['bonasa']),
 ('brain', ['brain']),
 ('butterfly', ['butterfly']),
 ('camera', ['camera']),
 ('cannon', ['cannon']),
 ('cellphone', ['cellphone']),
 ('chair', ['chair']),
 ('crab', ['crab']),
 ('crayfish', ['crayfish']),
 ('crocodile', ['crocodile']),
 ('cup', ['cup']),
 ('dalmatian', ['dalmatian']),
 ('dragonfly', ['dragonfly']),
 ('elephant', ['elephant']),
 ('ewer', ['ewer']),
 ('ferry', ['kerry']),
 ('flamingo', ['flamingo']),
 ('hedgehog', ['hedgehog']),
 ('kangaroo', ['kangaroo']),
 ('lamp', ['lamp']),
 ('laptop', ['laptop']),
 ('llama', ['llama']),
 ('lobster', ['lobster']),
 ('nautilus', ['nautilus']),
 ('panda', ['panda']),
 ('pizza', ['pizza']),
 ('platypus', ['platypus']),
 ('revolver', ['revolver']),
 ('saxophone', ['saxophone']),
 

In [78]:
missing_classes

['faces_easy',
 'motorbikes',
 'anchor',
 'brontosaurus',
 'buddha',
 'car_side',
 'ceiling_fan',
 'chandelier',
 'cougar_body',
 'cougar_face',
 'crocodile_head',
 'dollar_bill',
 'dolphin',
 'electric_guitar',
 'emu',
 'euphonium',
 'flamingo_head',
 'garfield',
 'gerenuk',
 'gramophone',
 'grand_piano',
 'hawksbill',
 'headphone',
 'helicopter',
 'ibis',
 'inline_skate',
 'joshua_tree',
 'ketch',
 'lotus',
 'mandolin',
 'mayfly',
 'menorah',
 'metronome',
 'minaret',
 'octopus',
 'okapi',
 'pagoda',
 'pigeon',
 'pyramid',
 'rhino',
 'rooster',
 'scissors',
 'snoopy',
 'soccer_ball',
 'stegosaurus',
 'stop_sign',
 'sunflower',
 'water_lilly',
 'wheelchair',
 'wild_cat',
 'windsor_chair',
 'yin_yang']

In [88]:
print(f"Missing classes: {len(missing_classes) / len(classes_caltech101) * 100:2f}% ({len(missing_classes)}/{len(classes_caltech101)})")

Missing classes: 51.485149% (52/101)


### Load CalTech101 Dataset

In [107]:
# TODO: download data from kaggle https://www.kaggle.com/datasets/imbikramsaha/caltech-101
import kagglehub

path = kagglehub.dataset_download("imbikramsaha/caltech-101")

100%|████████████████████████████████████████████████████████████████████| 131M/131M [00:04<00:00, 27.5MB/s]

Extracting files...


In [108]:
path

'/Users/emulie/.cache/kagglehub/datasets/imbikramsaha/caltech-101/versions/1'

### VGG19 performance on CalTech101 dataset

In [ ]:
vgg19 = torchvision.models.vgg19(weights='DEFAULT')

In [ ]:
# TODO

### Fine-Tuning VGG19 model

In [93]:
vgg19

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [100]:
class VGG19_Caltech(nn.Module):
    
    def __init__(self, num_classes=101):
        super(VGG19_Caltech, self).__init__()
        self.num_classes = num_classes

        self.features = vgg19.features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

In [101]:
model = VGG19_Caltech()

In [ ]:
# --- TODO: training model with caltech data

In [ ]:
# --- TODO: compare vgg19 performance vs vgg19 performance